In [146]:
import pandas as pd
import numpy as np
import math

In [211]:
Train_ratings=pd.read_csv("train_data.csv",encoding="ISO-8859-1")
Test_ratings = pd.read_csv("test_data.csv",encoding="ISO-8859-1")

In [212]:
print(Train_ratings.head(10))
print(Test_ratings.head(10))

                   userID              businessID  rate
0  ---1lKK3aKOuomHnwAkAow  -ErwgUmZ1-jHW_rSu55jlg   5.0
1  ---1lKK3aKOuomHnwAkAow  1JgaRBX0oiRsvEhHF3ZMjw   1.0
2  ---1lKK3aKOuomHnwAkAow  2BbFeotL85cIaBjSq1SWiA   1.0
3  ---1lKK3aKOuomHnwAkAow  5cbsjFtrntUAeUx51FaFTg   1.0
4  ---1lKK3aKOuomHnwAkAow  5rxJpTkeJa5rxMvL2NbSnQ   5.0
5  ---1lKK3aKOuomHnwAkAow  78TC3sZSYBzBsSJ0z5pyhw   1.0
6  ---1lKK3aKOuomHnwAkAow  Gaasy9YbPGVc8KcXcAIqEw   5.0
7  ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA   5.0
8  ---1lKK3aKOuomHnwAkAow  OnnvNs2JJ-B2xbwHEHbWDw   5.0
9  ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g   2.0
                   userID              businessID  rate
0  ---1lKK3aKOuomHnwAkAow  Qy_tDaVTWlS14fEglzo1Tg     4
1  ---1lKK3aKOuomHnwAkAow  bzPrrbDHRxstf9FSPw1Xjw     5
2  ---1lKK3aKOuomHnwAkAow  hk4NPN6W2xt30Cp-Fiqtuw     4
3  ---1lKK3aKOuomHnwAkAow  rq5dgoksPHkJwJNQKlGQ7w     5
4  ---1lKK3aKOuomHnwAkAow  ttH3ZbUcncBRIXqT-YVPCg     5
5  --CIuK7sUpaNzalLAlHJKA  sk0stgY4NDJYOX1MbNJ3P

In [214]:
Busin_mean= Train_ratings.groupby(['businessID'], as_index = False, sort = False).mean().rename(columns = {'rate': 'rating_mean'})[['businessID','rating_mean']]


In [215]:
User_mean =  Train_ratings.groupby(['userID'], as_index = False, sort = False).mean().rename(columns = {'rate': 'rating_mean'})[['userID','rating_mean']]

In [216]:
train_data = Train_ratings
print(train_data.head(10))

                   userID              businessID  rate
0  ---1lKK3aKOuomHnwAkAow  -ErwgUmZ1-jHW_rSu55jlg   5.0
1  ---1lKK3aKOuomHnwAkAow  1JgaRBX0oiRsvEhHF3ZMjw   1.0
2  ---1lKK3aKOuomHnwAkAow  2BbFeotL85cIaBjSq1SWiA   1.0
3  ---1lKK3aKOuomHnwAkAow  5cbsjFtrntUAeUx51FaFTg   1.0
4  ---1lKK3aKOuomHnwAkAow  5rxJpTkeJa5rxMvL2NbSnQ   5.0
5  ---1lKK3aKOuomHnwAkAow  78TC3sZSYBzBsSJ0z5pyhw   1.0
6  ---1lKK3aKOuomHnwAkAow  Gaasy9YbPGVc8KcXcAIqEw   5.0
7  ---1lKK3aKOuomHnwAkAow  HZdtHOEaKUL2SlWj5owgCA   5.0
8  ---1lKK3aKOuomHnwAkAow  OnnvNs2JJ-B2xbwHEHbWDw   5.0
9  ---1lKK3aKOuomHnwAkAow  PT9BRvRWx9kpcu02VouW1g   2.0


In [219]:
distinct_train_busin=np.unique(train_data['businessID'])
distinct_train_busin.sort()
print(len(distinct_train_busin))

162851


In [221]:
busin_sim = pd.DataFrame()
for busi1 in distinct_train_busin:
    busi_data1 = train_data[train_data["businessID"] == busi1]
    busi_data1 = busi_data1[['userID','businessID','rate']].drop_duplicates()
    busi_data1=busi_data1.rename(columns={'rate':'rate1'})
    busi_data1=busi_data1.rename(columns={'businessID':'businessID1'})
   
    for busi2 in distinct_train_busin:
        if busi1 >= busi2:
            continue
        busi_data2 = train_data[train_data["businessID"] == busi2]
        busi_data2 = busi_data2[['userID','businessID','rate']].drop_duplicates()
        busi_data2=busi_data2.rename(columns={'rate':'rate2'})
        busi_data2=busi_data2.rename(columns={'businessID':'businessID2'})
        
        busin_data_merge = pd.merge(busi_data1,busi_data2[['userID','businessID2','rate2']],on = 'userID', how = 'inner', sort = False)
        if len(busin_data_merge) == 0:
            continue;
        rate1_mean = busin_data_merge["rate1"].mean()
        rate2_mean = busin_data_merge["rate2"].mean()
        busin_data_merge["rate1"] = busin_data_merge["rate1"] - rate1_mean
        busin_data_merge["rate2"] = busin_data_merge["rate2"] - rate2_mean

        busi_val=np.sqrt(np.sum(np.square(busin_data_merge['rate1']), axis=0))
        busi_val2=np.sqrt(np.sum(np.square(busin_data_merge['rate2']), axis=0))
        busin_data_merge['vector_product']=(busin_data_merge['rate1']*busin_data_merge['rate2'])
        busin_data_merge= busin_data_merge.groupby(['businessID1','businessID2'], as_index = False, sort = False).sum()
        
        if (busi_val*busi_val2) == np.float64(0.0):
            busin_data_merge['dot'] = np.float64(0.0)
        else:
            busin_data_merge['dot']=busin_data_merge['vector_product']/(busi_val*busi2_val)
        busin_sim = busin_sim.append(busin_data_merge, ignore_index=True)
busin_sim =  busin_sim[busin_sim['dot']!=0]


KeyboardInterrupt: 

In [201]:
distinct_test_busin = np.unique(Test_ratings['businessID'])
distinct_test_busin

array([2, 3, 4, 7])

In [209]:
predicted = pd.DataFrame()
for test_busin in distinct_test_busin:
    test_data = Test_ratings[Test_ratings["businessID"] == test_busin]
    distinct_test_users = np.unique(test_data['userID'])
    test_busin_sim = busin_sim[(busin_sim["businessID1"] == test_busin)]
    test_busin_sim2 = busin_sim[(busin_sim["businessID2"] == test_busin)]
    test_busin_sim = test_busin_sim.append(test_busin_sim2)
    for test_user in distinct_test_users:
        print(str(test_user) + ", " + str(test_busin))
        test_user_busi_rate = Test_ratings[(Test_ratings["userID"] == test_user) & (Test_ratings["businessID"] == test_busin)]
        
        train_data_of_test_user = train_data[train_data["userID"] == test_user]
        if len(train_data_of_test_user) == 0:
            busin_mean = Busin_mean[Busin_mean["businessID"] == test_busin]
            print(busin_mean["rating_mean"])
            if len(busin_mean) == 0:
                predicted = predicted.append(test_user_busi_rate)
                continue
            else:
                new_rate = busin_mean["rating_mean"]
                print(new_rate)
                test_user_busi_rate["rate"] = new_rate
                predicted = predicted.append(test_user_busi_rate)
                continue
        other_busin_rated_by_user = np.unique(train_data_of_test_user["businessID"])
        test_user_busi_sim = pd.DataFrame()
        for other_busin in other_busin_rated_by_user:
            if other_busin < test_busin:
                this_couple = test_busin_sim[(test_busin_sim["businessID1"] == other_busin) & (test_busin_sim["businessID2"] == test_busin)]
                if len(this_couple) > 0:
                    this_couple["target"] = test_busin
                    this_couple["businessID"] = other_busin
                    this_couple = this_couple[["target","businessID","dot"]]
                    test_user_busi_sim = test_user_busi_sim.append(this_couple)
            elif other_busin > test_busin:
                this_couple = test_busin_sim[(test_busin_sim["businessID1"] == test_busin) & (test_busin_sim["businessID2"] == other_busin)]
                if len(this_couple) > 0:
                    this_couple["target"] = test_busin
                    this_couple["businessID"] = other_busin
                    this_couple = this_couple[["target","businessID","dot"]]
                    test_user_busi_sim = test_user_busi_sim.append(this_couple)
        
        k=3
        if len(test_user_busi_sim) == 0:
            new_rate = User_mean[User_mean["userID"] == test_user]["rating_mean"]
            test_user_busi_rate["rate"] = new_rate
            predicted = predicted.append(test_user_busi_rate)
            continue
        else:
            k = min(k,len(test_user_busi_sim))
            test_user_busi_sim = test_user_busi_sim.sort_values(['dot'], ascending=False)
            test_user_busi_sim = test_user_busi_sim.head(k)
            combine_data = test_user_busi_sim.merge(train_data_of_test_user,on = 'businessID', how = 'left', sort = False)
            combine_data["predicted_rate"] = (combine_data["dot"] * combine_data["rate"])
            combine_data['dot_abs']=combine_data['dot'].abs()

            combine_data = combine_data[['target','predicted_rate','dot_abs']]
            
            combine_data = combine_data.groupby(['target'], as_index = False, sort = False).sum()
            combine_data["rate"] = combine_data["predicted_rate"]/combine_data["dot_abs"]
            new_rate = combine_data["rate"][0]
            test_user_busi_rate["rate"] = new_rate
            print(test_user_busi_rate)
            predicted = predicted.append(test_user_busi_rate)
print(predicted)

            

2, 2
   userID  businessID      rate
7       2           2 -1.721225
5, 2


/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

   userID  businessID      rate
8       5           2  2.930097
13, 3
1    3.0
Name: rating_mean, dtype: float64
1    3.0
Name: rating_mean, dtype: float64
1, 4
   userID  businessID      rate
0       1           4 -1.498136
4, 4


/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

   userID  businessID      rate
1       4           4  1.512085
6, 4
   userID  businessID      rate
2       6           4 -1.347537
7, 4
   userID  businessID    rate
3       7           4 -0.3451
9, 4
   userID  businessID      rate
4       9           4 -4.379271
10, 4
   userID  businessID      rate
5      10           4 -1.103643
12, 4
   userID  businessID      rate
6      12           4  4.043561
9, 7
13, 7
Series([], Name: rating_mean, dtype: float64)
    userID  businessID      rate
7        2           2 -1.721225
8        5           2  2.930097
9       13           3       NaN
0        1           4 -1.498136
1        4           4  1.512085
2        6           4 -1.347537
3        7           4 -0.345100
4        9           4 -4.379271
5       10           4 -1.103643
6       12           4  4.043561
10       9           7       NaN
11      13           7  5.000000


/Users/dwj/miniconda3/envs/rec_sys/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [210]:
predict_truth = Test_ratings.merge(predicted, on = ('userID','businessID'), how = 'left', sort = False)
predict_truth['MSE'] = (predict_truth['rate_x'] - predict_truth['rate_y'])*(predict_truth['rate_x'] - predict_truth['rate_y'])
RMSE =np.sqrt(np.sum(predict_truth['MSE'])) / len(predict_truth)
print(RMSE)


1.1692806291070255
